In [27]:
#Import Necessary libraries
library('tidyverse')
library('dplyr')
library('forcats')
library(quantregForest)
library('kableExtra')
source('functions.R')

Loading required package: RColorBrewer



In [64]:
kFoldForest = function(Kfold, seed, datafr)
{ set.seed(seed)
  n = nrow(datafr)
  iperm<<-sample(n) # set as global for debugging check
  nhold = round(n/Kfold)
  reg = list()
  pred = list() 
  scoreVar = list()
  rocVar = list()
  pred_y = sample(n-nhold)
  for(k in 1:Kfold){ 
        ilow = (k-1)*nhold+1
        ihigh = k*nhold
        if(k==Kfold) { ihigh = n }
        ifold = iperm[ilow:ihigh]
      
        holdo = datafr[ifold,]
        train = datafr[-ifold,]
        train$price = log(train$price)
      
        qRF = quantregForest(train[,-1],train[,1],ntree = 50)
        predRF = predict(qRF, what=c(.1,.25,.5,.75,.9), newdata=holdo[,-1])
        preds50 = cbind(exp(predRF[,3]),exp(predRF[,2]),exp(predRF[,4]))
        preds80 = cbind(exp(predRF[,3]),exp(predRF[,1]),exp(predRF[,5]))
      
        IS50qRF = intervalScore(preds50,holdo$price,0.5)
        IS80qRF = intervalScore(preds80,holdo$price,0.8)
          
        outqRF = rbind(IS50qRF$summary,IS80qRF$summary) 
        colnames(outqRF) = c("level","avgleng","IS","cover") 
        print(outqRF)
        
  }
}

In [65]:
kFoldForest(Kfold = 3,seed = 123,datafr = train)

     level    avgleng        IS     cover
[1,]   0.5   53888.14  102749.5 0.2580705
[2,]   0.8 5152810.34 5192533.0 0.7855923
     level   avgleng        IS     cover
[1,]   0.5  176578.9  214651.2 0.4349979
[2,]   0.8 5453880.2 5481181.2 0.8616139
     level   avgleng        IS     cover
[1,]   0.5  173495.5  222781.2 0.2905940
[2,]   0.8 8756312.5 8791900.1 0.8177847
